# Data collection: Descriptive Statistics
* Descriptives of data collected during training and testing phase.
* Preparation of testing phase data set.

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from urllib.parse import urlparse
import ast
import re
import tldextract

In [ ]:
import WebSearcher as ws
ws.__version__

In [ ]:
PATH = "immigration/backup/"
out_path = "immigration/"

### 1) Data collection descriptives

In [ ]:
data = pd.read_csv(PATH+"data_test.txt", sep="\t")
data.shape

In [ ]:
rec2ins = pd.read_csv(PATH+"recipe2instance.txt", sep="\t")
rec2ins.shape

In [ ]:
recipe = pd.read_csv(PATH+"recipe.txt", sep="\t", lineterminator="\n")#, engine="python")
recipe.shape

In [ ]:
run = pd.read_csv(PATH+"run.txt", sep="\t")
run = run.rename(columns={'uid': 'run_uid'})
run.shape

In [ ]:
recipestep = pd.read_csv(PATH+"recipestep.txt", sep="\t")
html_steps = recipestep[recipestep.value=='innerHTML'].uid.to_list()
autocomplete_steps = recipestep[recipestep.type=='get_texts'].uid.to_list()
recipestep.shape

In [ ]:
def get_test_round(name):
    parts = name.split("_")
    test_num = parts[0].strip('test')
    round_num = parts[1]
    #print(test_num, round_num)
    return int(test_num), int(round_num)

def get_group(name):
    parts = name.split("_")
    if name.startswith('test'):
        group_num = parts[2]
    else:
        group_num = parts[1]
    return group_num

def get_conditions(name):
    parts = name.split('_')
    if name.startswith('train'):
        search_history = parts[3]
        user_input = parts[4][:-4]
        return search_history, user_input
    elif name.startswith('test'):
        search_history = parts[4]
        user_input = parts[5][:-4]
        return search_history, user_input
    else:
        return 'unknown', 'unknown'

def get_train_test(name):
    if name.startswith('test'):
        return 'test'
    elif name.startswith('train'):
        return 'train'
    else:
        return 'unknown'

In [ ]:
rec2ins[['test_num', 'round_num']] = rec2ins[rec2ins.new_name.str.startswith('test')]['new_name'].apply(lambda x: pd.Series(get_test_round(x)))
rec2ins['group'] = rec2ins['new_name'].apply(get_group)
rec2ins['train_test'] = rec2ins['new_name'].apply(get_train_test)
rec2ins[['search_history', 'user_input']] = rec2ins['new_name'].apply(lambda x: pd.Series(get_conditions(x)))

In [ ]:
keep_cols = [c for c in rec2ins.columns if c not in ["created", "uid"]]
run = pd.merge(run, rec2ins[keep_cols], how='left', on=['recipe_uid', 'instance_uid'])
run.shape

In [ ]:
# First round of data collection accidentally collected some agents multiple times (see logbook immigration). 
#Therefore, select first test row for every agent.
test_run = run[run.train_test=='test'].sort_values('created').drop_duplicates(['new_name'], keep='first')
train_run = run[run['train_test'] != 'test']
run = pd.concat([test_run, train_run])

print('Number of runs per round:')
run[run.train_test=='test'].round_num.value_counts(dropna=False)

In [ ]:
print('Total number of runs:', len(run))
run.train_test.value_counts(dropna=False)

In [ ]:
# each agent trained on average how many times?
print('Average number of training runs per agent:')
len(run[run.train_test=='train']) / run[run.train_test=='train'].new_name.nunique()

In [ ]:
print('Number of successful/unsuccessful runs:')
run.groupby('train_test').status.value_counts(dropna=False)
# 24 errors in training phase.

In [ ]:
print('Number of (un)successful runs per group:')
run.groupby(['group'])['status'].value_counts(dropna=False)
# does not occur only in one group.

In [ ]:
run['created'] = pd.to_datetime(run.created)
print('Training data collected between', run[run.train_test=='train'].created.min(), 'and', run[run.train_test=='train'].created.max())
print('Test data collected between', run[run.train_test=='test'].created.min(), 'and', run[run.train_test=='test'].created.max())

In [ ]:
dmin = run[run.round_num==1].created.min()
dmax = run[run.round_num==1].created.max()
print('Test data round 1 collected between', dmin, 'and', dmax, ', Timedelta:', dmax-dmin)
dmin = run[run.round_num==2].created.min()
dmax = run[run.round_num==2].created.max()
print('Test data round 2 collected between', dmin, 'and', dmax, ', Timedelta:', dmax-dmin)
dmin = run[run.round_num==3].created.min()
dmax = run[run.round_num==3].created.max()
print('Test data round 3 collected between', dmin, 'and', dmax, ', Timedelta:', dmax-dmin)

In [ ]:
data = pd.merge(data, run.drop('created' ,axis=1), on='run_uid', how='left')
data = data[~data.new_name.isna()] # drop those removed first round data collections 
data.shape

In [ ]:
#data.loc[(data.value!=np.nan)&(data.value.str.startswith("<")),'html'] = 1
html = data[data.step_uid.isin(html_steps)]
print('HTML unsuccessfully collected in', html.value.isna().sum(), 'cases.')
print(len(html), 'htmls collected')

In [ ]:
sq_dct = {1:
       {'pro':'vluchtelingen',
       'anti':'asielzoekers',
       'neutral':'immigratie'},
       2:
       {'pro':'vluchtelingencrisis',
       'anti':'azc',
       'neutral':'imigranten'},
       3:
       {'pro':'vluchtelingenproblematiek',
       'anti':'criminaliteit onder asielzoekers',
       'neutral':'immigratiecijfers'}
      }

def clean_up_list(lst, test_num, user_input):
    #print(lst, test_num, user_input)
    remove = sq_dct[test_num][user_input]
    lst = [w for w in lst if (not pd.isna(w)) & (w != remove)]
    return lst

In [ ]:
# autocompletes - remove if missing AND if same as input query
autocompletes = data[data.step_uid.isin(autocomplete_steps)]
autocompletes = autocompletes.groupby('run_uid')['value'].agg(list).reset_index()
autocompletes = pd.merge(autocompletes, data.drop_duplicates('run_uid')[['run_uid', 'test_num', 'round_num', 'group', 'train_test','search_history', 'user_input']], on='run_uid', how='left')
autocompletes['autocompletes'] = autocompletes.apply(lambda x: clean_up_list(x['value'], x['test_num'], x['user_input']), axis=1)
autocompletes.drop(columns=["value"], inplace=True)

In [ ]:
autocompletes['ac_length'] = autocompletes['autocompletes'].apply(len)

In [ ]:
# write files
html.to_csv(out_path+"html_test.csv", index=False)
autocompletes.to_csv(out_path+"autocompletes_test.csv", index=False)

### 2) Parsing of SERPs

In [ ]:
## parsing SERP HTMLs
def parse_serp(run_uid, value):
    #soup = ws.load_soup(html)
    soup = ws.make_soup(value)
    parsed = ws.parse_serp(soup)
    results = pd.DataFrame(parsed)
    
    if "sub_type" in results.columns: # not present in all parsed SERPs
        # make type distinction between two knowledge types: panel_rhs (knowledge panel) and featured snippet
        results.loc[results['type'] == 'knowledge', 'type'] = results['type'] + "_" + results['sub_type']
    
    # knowledge panel = rank -1
    results.loc[results['type'] == 'knowledge_panel_rhs', 'serp_rank'] = -1
    results.loc[results['type'] == 'knowledge_panel_rhs', 'cmpt_rank'] = -1

    # twitter cards: # header + after third card not visible on SERP
    mask = (results['type']=='twitter_cards')&((results['sub_type'] == 'header')|(results['sub_rank'] >= 4))
    results.loc[mask, 'not_visible'] = 1
    
    # videos: # after third not visible on SERP
    mask = (results['type']=='videos')&(results['sub_rank'] >= 3)
    results.loc[mask, 'not_visible'] = 1
    
    # add identifier
    results['run_uid'] = run_uid
    
    return results

In [ ]:
#test = html.iloc[0:100]
#test_results = []
#for run_uid, value in tqdm(zip(test['run_uid'], test['value'])):
#    #print(run_uid)
#    res = parse_serp(run_uid, value)
#    test_results.append(res)
#test_results = pd.concat(test_results)

In [ ]:
results = []
for run_uid, value in tqdm(zip(html['run_uid'], html['value'])):
    res = parse_serp(run_uid, value)
    results.append(res)
results = pd.concat(results)

In [ ]:
results.shape

In [ ]:
# select only rows that have reasonably been shown on the SERP.
# some unknown ones, e.g., "Resultaten zoeken voor...", "Vergelijkingssites"
print(results.type.value_counts())
results = results[results['not_visible']!=1].copy()
print(results.type.value_counts(), results.shape)

In [ ]:
# Reduce images to one row each.
images_rows = results[results['type']=='images'].groupby('run_uid').first().reset_index()
results = pd.concat([images_rows, results[results['type']!='images']])
results = results.sort_values(['run_uid', 'serp_rank'])

In [ ]:
# create subrank rows for people_also_ask and searches_related

In [ ]:
def expand_people_also_ask(row):
    details_list = row['details']
    new_rows=[]
    i=0
    for d in details_list:
        new_row = {'run_uid':row['run_uid'], 'type': row['type'], 'text': d, 'sub_rank': i, 'cmpt_rank':row['cmpt_rank'], 'serp_rank': row['serp_rank']}
        new_rows.append(new_row)
        i+=1
    return new_rows

In [ ]:
people_also_ask_rows = results[results['type']=='people_also_ask']
expanded_rows = pd.DataFrame(people_also_ask_rows.apply(expand_people_also_ask, axis=1).sum())
results = pd.concat([expanded_rows, results[results['type']!='people_also_ask']])
results = results.sort_values(['run_uid', 'serp_rank'])

In [ ]:
def expand_searches_related(row):
    details_list = row['details']
    new_rows=[]
    i=0
    for d in details_list:
        text = d['text']
        new_row = {'run_uid':row['run_uid'], 'type': row['type'], 'text': text, 'sub_rank': i, 'cmpt_rank':row['cmpt_rank'], 'serp_rank': row['serp_rank']}
        new_rows.append(new_row)
        i+=1
    return new_rows

In [ ]:
searches_related_rows = results[results['type']=='searches_related']
expanded_rows = pd.DataFrame(searches_related_rows.apply(expand_searches_related, axis=1).sum())
results = pd.concat([expanded_rows, results[results['type']!='searches_related']])
results = results.sort_values(['run_uid', 'serp_rank'])

In [ ]:
def get_snippet_details(row):
    details = row['details']
    
    row['url'] = details['urls'][0]['url'].strip('.')
    row['title'] = details['heading']
    row['text'] = details['text']
    return row

In [ ]:
results.loc[results['type'] == 'knowledge_featured_snippet'] = results.loc[results['type'] == 'knowledge_featured_snippet'].apply(get_snippet_details,axis=1)

In [ ]:
def parse_domain(url):
    if isinstance(url, str):
        extracted = tldextract.extract(url)
        domain = extracted.domain + "." + extracted.suffix
        return domain
    else:
        return np.nan

In [ ]:
# parse domain from URL
results['domain'] = results['url'].apply(parse_domain)
results.shape

In [ ]:
# make sure empty strings are NaN values
results.loc[(results['domain']=='')|(results['domain']=='.'), 'domain'] = np.nan

In [ ]:
#results[~results.domain.isna()].domain.unique(), results[~results.domain.isna()].domain.nunique()

In [ ]:
# add domain categories
cats = pd.read_csv("news_categories_adjusted_domainlevel.csv")
print('Number of domains in categorisation file:', len(cats))
results = pd.merge(results, cats[~cats.domain.isna()], on=['domain'], how='left')
results.shape

In [ ]:
# non_core = missing final_category
results.loc[(results.final_category=='non_core'), 'final_category'] = np.nan
# fix avrotros mistake
results.loc[(results.final_category=='special_int'), 'final_category'] = 'inst'

In [ ]:
# share of rows with domains identified
#results[~results['domain'].isna()].final_category.value_counts(dropna=False)
results[~results['domain'].isna()].final_category.value_counts(dropna=False, normalize=True)

In [ ]:
#missingcat_domains = results[(~results['domain'].isna())&(results.final_category.isna())].domain.unique()
#pd.DataFrame(missingcat_domains, columns=['domain']).to_csv('missingcat_domains_im.csv')
#missingcat_domains

In [ ]:
# additional coding of missing domains
cats2 = pd.read_csv("finalcategory_coding.csv", sep=";")
cats2 = cats2.rename(columns={'final_category': 'final_category2'})
cats2.shape

In [ ]:
results = pd.merge(results, cats2, on='domain', how='left')
results.loc[(~results['domain'].isna())&(results.final_category.isna()), 'final_category'] = results['final_category2']
results.drop(columns=['final_category2'], inplace=True)

In [ ]:
# 100% coverage
results[(~results['domain'].isna())].final_category.value_counts(dropna=False)

In [ ]:
results[(~results['domain'].isna())].final_category.value_counts(dropna=False, normalize=True)

In [ ]:
def clean_video_title(row):
    # title is everything before the channel
    title = row['title']
    channel = row['cite']
    cleaned_title = title.split(channel)[0].strip()
    ## Remove YouTube manually
    #cleaned_title = cleaned_title.replace('YouTube', '')
    row['title'] = cleaned_title
    return row

In [ ]:
results.loc[results['type'] == 'videos'] = results.loc[results['type'] == 'videos'].apply(clean_video_title, axis=1)

In [ ]:
## reset serp_rank (necessary given rows expanded).
results = results.sort_values(['run_uid', 'serp_rank', 'sub_rank'])
results['serp_rank'] = results.groupby('run_uid').cumcount() + 1
# knowledge panel = rank -1
results.loc[results['type'] == 'knowledge_panel_rhs', 'serp_rank'] = -1
results.loc[results['type'] == 'knowledge_panel_rhs', 'cmpt_rank'] = -1

In [ ]:
results.to_csv(out_path+"result_data_test.csv", index=False)